<a href="https://colab.research.google.com/github/laura-turnbull-lloyd/STDH_teaching/blob/main/Lecture_4_analysing_spatial_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introduction

Today we're going to be focussing on how to assess the spatial distribution of hazards, using landslides as an example.

You'll be working with a spatial dataset consisting of polygons representing landslides that occurred during the 2008 Mw 7.9 Wenchuan earthquake in China.

The data represent an area of 20 x 20 km.


# 2. Packages we'll be working with

The main package we'll be working with is `GeoPandas`, which offers a pandas-like interface to working with geodata. Think of this as your tool for basic data manipulation and transformation, much like pandas.

The other packages listed are those that you've worked with before, or packages that geopandas depends on.

Let's load the packages:

In [ ]:
# Colab environment (run once)
!pip -q install "geopandas>=0.14" "shapely>=2.0" "pyproj>=3.6" "rtree>=1.3.0" "seaborn>=0.13" "pointpats"


In [ ]:
import numpy as np, pandas as pd, geopandas as gpd, matplotlib.pyplot as plt, seaborn as sns, requests, zipfile, io
from shapely.geometry import Point, box, Polygon
# to make sure that pandas outputs values in an easy to read format, we can customise the output format:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
from pointpats import (
    distance_statistics,
    QStatistic,
    random,
    PointPattern
)


# 3. Reading in landslide data
The data have been provided for you in a shapefile format, in a zipped up file for you on GitHub. In the script below, you will automatically extract the data from GitHub (so no need to download it).

In case you also wish to explore the data in a more traditional Geographical Infromation System (e.g. ArcPro or QGIS), I have also provided the data for you on Learn Ultra on this weeks page.


To read in the data from GitHub:

In [ ]:
# read in data that's provided for you on Github
url = "https://github.com/laura-turnbull-lloyd/STDH_teaching/raw/main/landslides_clip.zip"

# Download the zip file into memory
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))

# read in the shapefile
z.extractall("landslides_clip")
LS = gpd.read_file("landslides_clip") # this reads in the landslide data

To view the first few lines of the data:

In [ ]:


# View the header info of the landslide data we've just read in
print(LS.head())


Alternatively we can print out the whole dataset which will show us the first and last few lines in the dataset:

In [ ]:
#Printing out the whole dataset
print(LS)

You can see that there are 6604 rows in the data, which means there are 6604 individual landslide polygons.

Note that python is a bit wierd in that it starts its index numbering from zero.

Each row in the landslide dataset contains a unique identifier (in column ID), the area of the polygon in $m^2$ (area_rob), the initials of the person who mapped it (data), and the geometry of the data (geometry). These attributes are linked to each individual landslide polygon.

Using the `gpd.read_file`command, you read the data into a 'geodataframe', which is very similar to a traditional, non-spatial pandas DataFrame, but with an additional column called geometry.

# 4. Determining the coordinate system of the data
It's important to be aware of the coordinate system of the data you're working with. You can find it out using the ```crs``` command. It's easy to make mistakes in spatial data analysis when you aren't aware of the coordinate system of your data, and if it's an unsuitable coordinate system for the type of analysis you want to undertake.



In [ ]:
LS.crs

# 5. Plotting spatial data
We can plot the raw spatial data using the code below. In this code, we undertake the following steps:
1. Create a figure named `f` with one axis named `ax` by using the command plt.subplots (part of the library matplotlib, which we have imported at the top of the notebook).
Note how the method is returning two elements and we can assign each of them to objects with different name (`f` and `ax`) by simply listing them at the front of the line, separated by commas.
2. We plot the geographies and tell the function that we want it to draw the polygons on the axis we are passing, `ax`. This method returns the axis with the geographies in them, so we make sure to store it on an object with the same name, `ax`.
3. We draw the entire plot by calling `plt.show()`.

For more information on matplotlib plotting conventions, see [here](https://matplotlib.org/).

You can also save figures outside of this notebook, using the ```plt.savefig``` command -- see below for an example. Any figures you save will be saved in the current working directory and therefore will need to be moved if you want to access them after the session has ended.



In [ ]:
# Setup figure and axis
f, ax = plt.subplots(1, figsize=(16, 8))
# Plot layer of polygons on the axis
LS.plot(ax=ax)
# Remove axis frames
#ax.set_axis_off()
# Display
plt.show()

# Save figure to a PNG file
plt.savefig('Wenchuan_landslides.png')

plt.tight_layout()

# For a very high resolution image we can add the dpi in the command, e.g.
#plt.savefig('Wenchuan_landslides.png', dpi = 1080) # I've left this line commented, as you don't need to run it for now.


# 6. Exploratory data analysis
It's always useful to undertake exploratory data analysis to establish key characteristics about the data (hazard) that we're working with.

We can use the pandas summary statistics (describe) function to be able to explore the data.

The describe funciton gives the:
> count

> mean

> standard deviation

> minimum

> 25 percentile

> 50 percentile

> 75 percentiles

> max

of all numeric columns in the dataset


In [ ]:
LS.describe()

**What's the maximum landslide size (including correct units)?**



For other summary infromation that is not given by the describe function, we can calculate this manually. For instance, we might be interested in the total area of landslides in the study area. We can calculate this using the pandas 'series sum' function.

In [ ]:
total_area = pd.Series(LS['area_rob'].sum(), index=['area_rob'])
print(total_area)

**What's the total area affected by landslides in the study area?**

**What percentage of the entire study area is affected by landslides?**
To answer this question you will have to undertake a further calculation which you can enter below:




In [ ]:
# Add your calculations here in order to answer the above question:

To see how I did it, you can click 'show code' below, but do have a go at figuring this out for yourself first!

In [ ]:
# @title
study_area = 20000*20000
percent_LS = total_area/study_area*100
percent_LS

# 7. Point pattern analysis: testing if there is a spatial pattern

We can begin by asking a fundamental question: **Are landslides randomly distributed across the study area, or do they show spatial patterning?**

To be able to use the toolbox of point pattern analysis answer this question, we first need to represent the data (in this case, landslides) as points.

To do this we extract a point (the centroid) from the landslide polygons.

In [ ]:
#extracting points from the landslide polygons
points = LS.copy()
# change geometry
points['geometry'] = points['geometry'].centroid
points

As we've already established, the first step to get a sense of what the spatial dimension of this dataset looks like is to plot it.

In [ ]:
f1, ax = plt.subplots(1, figsize=(12, 8))
points.plot(ax=ax) # this is the landslide data represented as points
plt.autoscale(True)
plt.tight_layout()

Now that we've got the landslide data in a point format, we can test this observed point pattern against **complete spatial randomness (CSR)**, where there is neither clustering nor dispersion.

We can do this by using the ```pointspats``` package to simulate CSR from a given point set, using the ```pointpats.random``` module, which creates a random dataset, based on the landslide dataset (i.e., it shares the same geomoetry and has the same number of points), that exhibits complete spatial randomness:


In [ ]:
#create CSR from the point data set

# first, we extract the x and y coordinates of each point
points["x"] = points.geometry.x
points["y"] = points.geometry.y

# then we create a list of coordinates
coordinates = points[["x", "y"]].values

# then we shuffle everything up
random_pattern = random.poisson(coordinates, size=len(coordinates))

Let's compare the random pattern with the
observed landslide pattern.

In [ ]:
f2, ax = plt.subplots(1, figsize=(9, 9))
plt.scatter(
    *coordinates.T,
    color="k",
    marker=".",
    label="Observed landslides"
)
plt.scatter(*random_pattern.T, color="r", marker="x", label="Random")
ax.legend(ncol=1, loc="right")
plt.show()

Now we're going to test whether the landslide data exhibit a spatial pattern or not, by dividing the region into a grid of equal-size quadrats (cells) and then counting how many landslides fall in each cell.

By examining whether observations are spread evenly over cells, this quadrat approach aims to estimate whether points are spread out, or if they are clustered into a few cells.

In python, using the pointpats package, we can visualize the point data, and how the points are counted within each grid cell. We can do this using the  ``QStatistic.plot()`` method.

In [ ]:
qstat = QStatistic(coordinates, nx=15, ny=15) # nx and ny are the number of cells in the x and y directions
qstat.plot()

Here, for a 15 x 15 grid (note that this is specified in the code block above) spanning the point pattern, we can see a mix of very high values and very low values.

We can use the $X^2$ test statistic to test if the pattern is random or not, which is expressed as:
$$
\chi^2 = \sum_{i=1}^{k} \frac{(O_i - E_i)^2}{E_i}
$$
where $O_i$ is the observed count in quadrat 𝑖, $E_i$ is the expected count under CSR (same overall intensity), and $k$ is the number of quadrats.

If  $X^2$ is small and the p-value is large (p > 0.05), the pattern is consistent with CSR.

If  $X^2$ is large and p < 0.05, the pattern departs from randomness, suggesting clustering or regular spacing.

At this point, the test tells us whether the pattern is clustered, but not where or by how much.

Let's have a look at the $X^2$ statistics for the landslide point dataset...


In [ ]:
print("chi2 p value", qstat.chi2_pvalue)
print("chi2 value", qstat.chi2)

We have a significant P-value and a very high $X^2$ statistic, telling us that the landslide data significantly deviate from CSR.  

Now, let's repeat the analysis for the randomly distributed dataset that we generated, and view the resulting stats:

In [ ]:
qstat_rand = QStatistic(random_pattern, nx=15, ny=15)
qstat_rand.plot()
print("chi2 p value", qstat_rand.chi2_pvalue)
print("chi2 value", qstat_rand.chi2)

**What does this analysis show? Can you think of any potential issues with the above analysis?**

**Take home message:** At this point, this analysis tells is us whether the pattern is clustered (across the whole study area), but not where or by how much.

# 8. Visualizing patterns more clearly: point density

We can extend the quadrat approach used in the previous section, where we counted the number of landslides in each cell, to view it as a spatial or 2-D histogram.

To do this from scratch, we generate a regular grid (either squared or hexagonal), and count how many points fall within each grid cell. This is attractive because it is simple and intuitive.

First, let's create the regular grid.

In [ ]:
# total area for the grid
xmin, ymin, xmax, ymax= LS.total_bounds # this function extracts the bounding coordinates of the landslide data
width = 500 # this sets the size of the grid cell width
height = 500 # this sets the size of the grid cell width
# projection of the grid - this sets the crs to be the same as the data we area already working with
crs = LS.crs
rows = int(np.ceil((ymax-ymin) /  height))
cols = int(np.ceil((xmax-xmin) / width))
XleftOrigin = xmin
XrightOrigin = xmin + width
YtopOrigin = ymax
YbottomOrigin = ymax- height
polygons = []
for i in range(cols):
   Ytop = YtopOrigin
   Ybottom =YbottomOrigin
   for j in range(rows):
       polygons.append(Polygon([(XleftOrigin, Ytop), (XrightOrigin, Ytop), (XrightOrigin, Ybottom), (XleftOrigin, Ybottom)]))
       Ytop = Ytop - height
       Ybottom = Ybottom - height
   XleftOrigin = XleftOrigin + width
   XrightOrigin = XrightOrigin + width

grid = gpd.GeoDataFrame({'geometry':polygons}, crs=crs)
grid

Next, we'll create a new object called `landslide_count` and store the result of a spatial join between landslide points and the grid.

In [ ]:
landslide_count = gpd.sjoin(grid, points, how='left', predicate ='contains')
landslide_count

In the above table, NaN means "Not a number", and this happens when the count function didn't count a landslide in a particular cell.

Each landslide carries the ID of its corresponding grid cell (on the far left of the above table). Let's explicitly add this grid cell ID to the geodataframe:

In [ ]:
landslide_count['grid_index'] = landslide_count.index
landslide_count

Now we want to count the number of landslides in each grid cell. We can do this using the dissolve tool (which simplifies data), counting the number of landslides within each grid cell.

In [ ]:
landslide_count_dissolve = landslide_count.dissolve(by='grid_index', aggfunc='count')
landslide_count_dissolve

In the table above, you can see that for the attributes that were associated with the landslide point data, you now have the 'count' of how many landslides there were in each grid cell.

So, for example, for grid cell with an index of 3, there were 7 landslide points within that cell. Whilst for grid cell with an index of 0, there were 0 landslides.

We can now plot out the resulting landslide point density map.

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 8))
landslide_count_dissolve.plot(ax=ax, column='index_right', cmap='jet', legend = True) # here, cmap = 'jet' specifies the colour ramp I've chosen to use.
plt.autoscale(True)
grid.plot(ax=ax, facecolor="none", edgecolor='grey') # this is where you overlay the grid

# 8. Visualizing patterns more clearly: kernel density estimate

We can also summarise the spatial distribution of points using a kernel density function. This is simple to achieve using the `seaborn package`.

A kernel density function replaces plotting every single point by estimating the continuous observed probability distribution. It's not too dissimilar to the point density counts that you've just carried out, but it differs in that it creates a surface that models the probability of point density over space.

The idea behind kernel density estimates (KDEs) is to count the number of points in a continuous way. Instead of using discrete counting, where you include a point in the count if it is inside a certain boundary and ignore it otherwise, KDEs use functions (kernels) that include points but give different weights to each one depending on how far from the location where we are counting the point is.

Creating a kernel density estimate is very straightfoward in Python. In its simplest form, we can run the following single line of code:

In [ ]:
sns.kdeplot(data = points, x = "x", y = "y", fill=True, cmap='viridis', cbar = True)

Much like with the bin size in the histogram, the ability of the kernel density estimate to accurately represent the data depends on the choice of the bandwidth, which essentially represents the radius of smoothing.

An over-smoothed estimate might erase meaningful patterns, but an under-smoothed estimate can obscure the true patterns within random noise.

The easiest way to check the robustness of the estimate is to adjust the default bandwidth (which is a backend calculation in seaborn using scipy, and is based on the distribution of the data).

To do this, we can add a bw_adjust function (```bw_adjust = 0.5```) to the code below, and have a play around to see how different values of bw_adjust alter the resulting kernel density estimation.

In [ ]:
sns.kdeplot(data = points, x = "x", y = "y", fill=True, cmap='viridis', cbar = True, bw_adjust = 0.5)

You can read more about the different parameter options for kernel density estimates using the seaborn package [here](https://seaborn.pydata.org/generated/seaborn.kdeplot.html).

In summary:

A KDE with a small bandwidth → fine detail, local patterns
*   Produces a highly detailed surface with many small peaks
*   Highlights local clusters and sharp changes in density
*   May appear noisy — individual landslides dominate the surface
*   Useful for identifying hotspots at smaller scales

A KDE with a larger bandwidth → broad smoothing, regional trends
*   Each landslide influences a larger area → generalized pattern
*   Emphasizes broader-scale patterns
*   Can mask local variability and small clusters

# 9. Area density measures

Point density measures treats every landslide equally, regardless of their size.
Often, we're keen to understand the overall magnitude of a hazard, not just whether or not an event has occurred. For hazards such as landslides and wildfires, the area (i.e. the footprint of the event) is a good place to start.

Similar to point density calculations, to undertake area density calculations, we need to use a grid to divide up the study area into cells. You already created a grid to use in the point density calculations, so we'll just use that same grid.

Plotting the grid over the data shows how the landslides are generally clustered into a few regions and we’ll end up with many “empty” grid cells:

In [ ]:
f3, ax = plt.subplots(1, figsize=(12, 8))
LS.plot(ax=ax) # this is the landslide data
plt.autoscale(True)
grid.plot(ax=ax, facecolor="none", edgecolor='grey') # this is where you overlay the grid!
plt.tight_layout()

We’re going to determine the area of landslides in each square polygon. This takes a couple of steps in `GeoPandas`.

First, we use the intersect tool to divide up the landslide polygons based on the grid cells.

Then we calculate the area of each fragmented landslide polygon, and add the landslide index to the dataframe (just to make sure it gets carried over into the subsequent analysis).

In [ ]:
#LS_intersection = LS.overlay(cell, keep_geom_type = False)
LS_intersect = LS.overlay(grid, how='intersection', keep_geom_type=False)
LS_intersect['LS_area'] =LS_intersect.apply(lambda row: row.geometry.area,axis=1) # calculate the area of each landslide segment
LS_intersect['LS_index'] = LS_intersect.index
print(LS_intersect)


We can plot out the data to check it looks sensible...

In [ ]:
f4, ax = plt.subplots(1, figsize=(25, 15))
LS_intersect.plot(ax=ax, facecolor = "none", edgecolor = 'red')
plt.autoscale(True)
plt.tight_layout()

This looks good. You can clearly see where landslide polygons have been divided up where they cross grid cells.

The next task is to undertake a spatial join, to join the landslide data to the grid. We do this using the geopandas spatial join, ```sjoin``` tool.



In [ ]:
merged = gpd.sjoin(grid, LS_intersect, predicate = 'contains', how='left')
merged

As we did before, we'll add the grid index to the dataframe:

In [ ]:
merged['grid_index'] = merged.index # here, we add the index of the cells covering the study area into the dataframe so that we can use this in our analysis

In [ ]:
print(merged)

In the dataframe you've just printed out, you can see that some landslides have the same grid index -- this is because there is more than one landslide in a signle grid cell. What we want is the total area of landslides in a single grid cell. To achieve this, we can dissolve the data, in which we combine information (landslide area) from individual landslide polygons, to generate a total landslide area per grid cell.

In [ ]:
# Compute stats per grid cell -- aggregate landslides to grid cells with dissolve
dissolve_merged = merged.dissolve(by='grid_index', aggfunc='sum') # sums up all the values in the area column
dissolve_merged

Next, to help with landslide area density calculations, we need the total area of each grid cell. There are multiple ways we can calculate this (we specified the dimensions of each grid cell earlier). I always like to calculate dimensions based on the actual data, as there's less room for error.

In [ ]:
dissolve_merged['cellarea'] =dissolve_merged.apply(lambda row: row.geometry.area,axis=1)# determine the area of each cell
dissolve_merged

Now you've got the area of landsides in each grid cell you can finally calculate the landslide area density, which is simply the area of landslides in each cell, divided by the area of that cell, multiplied by 100, to give landslide area as a %.


In [ ]:
dissolve_merged['area_density'] = (dissolve_merged['LS_area']/dissolve_merged['cellarea'])*100
dissolve_merged

Finally, to plot out the end product of this analysis:

In [ ]:
f5, ax = plt.subplots(1, figsize=(12, 8))
dissolve_merged.plot(ax=ax, column='area_density', cmap='jet', legend = True) # here, cmap = 'jet' specifies the colour ramp I've chosen to use.
plt.autoscale(True)
grid.plot(ax=ax, facecolor="none", edgecolor='grey') # this is where you overlay the grid


# 10. Compare the different approaches to assess spatial patterns of landslides

You've used point density, kernel density and area density measures to assess the spatial patterns of landslides.

Compare and contrast the resulting patterns of landslides from these different types of analyses.

Have a think about which approach is most useful and why, in consideration of different hazards you're interested in, and in consideration of different data formats available to us (e.g. point/polygon/line representations of hazards).

